# Ingest SEC DERA data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

Load Environment Variables

In [2]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [5]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'

In [6]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('default',)
('information_schema',)
('sandbox',)


Enter the Pandas!

In [7]:
import pandas as pd
import re

Drop previous tables and schema to start with a fresh slate

In [8]:
for view in [ 'assets_by_lei', 'assets_usd_by_lei', 'assets_xyz_by_lei',
              'cash_by_adsh_ddate', 'cash_by_lei', 'cash_usd_by_lei', 'cash_xyz_by_lei',
              'debt_by_adsh_ddate', 'debt_by_lei', 'debt_usd_by_lei', 'debt_xyz_by_lei',
              'financials_by_lei',
              'float_by_lei', 'float_usd_by_lei', 'float_xyz_by_lei',
              'fy_revenue_by_lei', 'fy_revenue_usd_by_lei', 'fy_revenue_xyz_by_lei',
              'fy_income_by_lei', 'fy_income_usd_by_lei', 'fy_income_xyz_by_lei',
            ]:
    sql = f"""
drop view if exists {ingest_catalog}.{ingest_schema}.{view}
"""
    # print(sql)
    qres = engine.execute(sql)
    # print(qres.fetchall())

Load `ticker` file (updated sporadically from https://www.sec.gov/include/ticker.txt)

In [9]:
ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'SEC-DERA/ticker.txt')
ticker_file.download_file(f'/tmp/dera-ticker.txt')
ticker_df = pd.read_csv(f'/tmp/dera-ticker.txt', names=['tname', 'cik'], header=None, sep='\t', dtype={'tname':'string','cik':'int64'}, engine='c')
ticker_dict = dict(zip(ticker_df.cik, ticker_df.tname))

ticker_df

,tname,cik
0,aapl,320193
1,msft,789019
2,goog,1652044
3,amzn,1018724
4,tsla,1318605
...,...,...
12416,afacu,1849489
12417,afaqw,1841661
12418,aesc,874761
12419,aesew,1708341


In [10]:
ingest_table = 'ticker'
columnschema = osc.create_table_schema_pairs(ticker_df)

tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
partitioning = array['bucket(tname,20)', 'bucket(cik,20)'],
format = 'ORC'
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())
ticker_df.to_sql(ingest_table,
                 con=engine, schema=ingest_schema, if_exists='replace',
                 index=False,
                 method=osc.TrinoBatchInsert(batch_size = 12000, verbose = True))


create table if not exists osc_datacommons_dev.sandbox.ticker(
    tname varchar,
    cik bigint
) with (
partitioning = array['bucket(tname,20)', 'bucket(cik,20)'],
format = 'ORC'
)

[(True,)]


/opt/app-root/lib64/python3.8/site-packages/pandas/io/sql.py:1787: SAWarning: index key 'record_count' was not located in columns for table 'ticker'
  self.meta.reflect(bind=self.connectable, only=[table_name], schema=schema)
/opt/app-root/lib64/python3.8/site-packages/pandas/io/sql.py:1787: SAWarning: index key 'file_count' was not located in columns for table 'ticker'
  self.meta.reflect(bind=self.connectable, only=[table_name], schema=schema)
/opt/app-root/lib64/python3.8/site-packages/pandas/io/sql.py:1787: SAWarning: index key 'total_size' was not located in columns for table 'ticker'
  self.meta.reflect(bind=self.connectable, only=[table_name], schema=schema)
/opt/app-root/lib64/python3.8/site-packages/pandas/io/sql.py:1787: SAWarning: index key 'data' was not located in columns for table 'ticker'
  self.meta.reflect(bind=self.connectable, only=[table_name], schema=schema)


inserting 12000 records
  ('aapl', 320193)
  ('msft', 789019)
  ('goog', 1652044)
  ...
  ('eqha-wt', 1826729)
constructed fully qualified table name as: "sandbox.ticker"
batch insert result: [(12000,)]
inserting 421 records
  ('eqh-pa', 1333986)
  ('eqh-pc', 1333986)
  ('ephyu', 1827248)
  ...
  ('aeva-wt', 1789029)
constructed fully qualified table name as: "sandbox.ticker"
batch insert result: [(421,)]


Prepare GLEIF matching data

In [11]:
gleif_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'mtiemann-GLEIF/DERA-matches.csv')
gleif_file.download_file(f'/tmp/dera-gleif.csv')
gleif_df = pd.read_csv(f'/tmp/dera-gleif.csv', header=0, sep=',', dtype=str, engine='c')
gleif_dict = dict(zip(gleif_df.name, gleif_df.LEI))

Load the SUB, NUM, and TAG tables into Trino

In [12]:
# import re
import uuid

# Borrowed/stole this definition from SEC Corp Financials notebook...
float_tags = [
    'EntityPublicFloat',
    'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest',
    'FreeFloat',
    'PublicFloat',
    'PublicFloatValue',
]

# These are in priority preference order
share_tags = [
    'EntityCommonStockSharesOutstanding',
    'CommonStockSharesOutstanding',
    'SharesOutstanding',
    'WeightedAverageNumberOfDilutedSharesOutstanding',
    'WeightedAverageNumberOfSharesOutstandingBasic',
]

# We don't want anything that reports par value, such as CommonStockValue
shareprice_tags = [
    'SharePrice',
    'PerSharePrice',
    'MarketValuePerShare',
    'SaleOfStockPricePerShare',
    'CashPricePerOrdinaryShare',
    'TreasurySharesValuePerShare',
    'SharesOutstandingPricePerShare',
]

treasury_share_tags = [
    'TreasuryStockShares',
    'TreasuryStockShares1',
]

treasury_value_tags = [
    'TreasurySharesMarketValue',
    'FairValueOfTreasuryShares',
    'MarketValueOfTreasuryShares',
]

all_float_helper_tags = share_tags + shareprice_tags + treasury_share_tags + treasury_value_tags

dera_regex = re.compile(r' ?/.*$')
dera_df = {}

from math import floor

def generate_intermediate_ddate(df_orig):
    if len(df_orig)==1:
        return df_orig
    year1 = df_orig.iloc[0].ddate
    year2 = df_orig.iloc[1].ddate
    if (year1-year2).days > 731:
        print("gap years")
        print(df_orig.iloc[0:2])
    new_df = df_orig.iloc[[0]].copy()
    year_end = pd.to_datetime(f"{floor((year1.year+year2.year)/2.0)}1231", format='%Y%m%d', utc=True)
    new_df.ddate = year_end
    if year1.year==year2.year:
        print("same years")
        print(df_orig.iloc[0:2])
        new_df.value = (df_orig.iloc[0].value + df_orig.iloc[1].value)/2.0
    else:
        new_df.value = ((365.0-(year1-year_end).days)*df_orig.iloc[0].value + (year_end-year2).days*df_orig.iloc[1].value)/365.0
    new_df.version = df_orig.iloc[0].adsh
    print(f"adding fact ({new_df.tag})")
    return new_df

# When this function is called, we already know that we have no matches in FLOAT_TAGS.
# GROUPED_DF is grouped by ADSH and only for annual reports.  DDATE can be anything (because many reports look back 1-5 years)
# We are working these annual reports quarter by quarter for the quarter in which they are reported

def infer_float(grouped_df):
    new_df = pd.DataFrame()
    for key, item in grouped_df:
        df = grouped_df.get_group(key)
        # We have no overall float value.  Build from shares outstanding * price
        df_shares = df[df.tag.isin(share_tags) & (df.value>0)]
        if df_shares.empty:
            continue
        else:
            for share_tag in share_tags:
                if not df_shares[df_shares.tag==share_tag].empty:
                    df_shares = df_shares[df_shares.tag==share_tag]
                    break
            if len(df_shares[df_shares.ddate.dt.year==df_shares.fy.dt.year])>1:
                print("thinning shares")
                df_shares = df_shares[df_shares.ddate.dt.year==df_shares.fy.dt.year]
                df_shares = df_shares.sort_values('ddate', ascending=False)
        df_prices = df[df.tag.isin(shareprice_tags)]
        # We have no overall price.  Build from price derived from treasury valuation
        if df_prices.empty:
            df_treasury_shares = df[df.tag.isin(treasury_share_tags)]
            df_treasury_value = df[df.tag.isin(treasury_value_tags)]
            if df_treasury_shares.empty or df_treasury_value.empty:
                continue
            df_svp = df_treasury_value.merge(df_treasury_shares, on=['adsh', 'ddate', 'coreg'])
            if df_svp.empty:
                print(f"{df.adsh.iat[0]}: merge failed (1)")
                continue
            # Pick latest date / largest number of shares as basis
            df_float = df_svp.sort_values(['ddate', 'value_y'], ascending=False).iloc[[0]].copy()
            price_per_share = df_float.value_x.squeeze() / df_float.value_y.squeeze()
            df_float.rename(columns={'uom_x':'uom'},inplace=True)
            tag = 'ComputedTreasuryFloat'
        else:
            # if df_prices[df_prices.tag.str.startswith('ShareBasedCompensationArrangementByShareBasedPaymentAward')].empty:
            #     print(f"Must use market prices; len(df_prices) =  {len(df_prices)}")
            # else:
            #     print(f"Can use Share Based Comp {df_prices.tag.str[45:]}:\n{df_prices}\n\n")
            # We derive a price from market reports
            df_svp = df_prices.merge(df_shares, on=['adsh', 'ddate', 'coreg'])
            if df_svp.empty:
                if len (df_prices[df_prices.ddate.dt.year==df_prices.fy.dt.year])>1:
                    print("thinning prices")
                    df_prices = df_prices[df_prices.ddate.dt.year==df_prices.fy.dt.year]
                if len(df_prices)<3 and len(df_shares)<3:
                    df_prices = generate_intermediate_ddate(df_prices.sort_values('ddate', ascending=False))
                    df_float = generate_intermediate_ddate(df_shares.sort_values('ddate', ascending=False))
                    price_per_share = df_prices.value.squeeze()
                    # print("merge rescued (2)")
                    # display(df_shares)
                else:
                    print(f"{df.adsh.iat[0]}: merge failed (2)")
                    print(f"len(df_prices) = {len(df_prices)}")
                    print(f"len(df_shares) = {len(df_shares)}")
                    display(df_prices)
                    display(df_shares)
                    continue
            else:
                # Pick latest date / largest number of shares as basis
                df_float = df_svp.sort_values(['ddate', 'value_y'], ascending=False).iloc[[0]]
                price_per_share = df_float.value_x.squeeze() # value_x is a price in this case
                df_float.rename(columns={'uom_x':'uom'},inplace=True)
            tag = 'ComputedMarketFloat'
        df_float = df_float[['adsh', 'ddate', 'uom', 'coreg']]
        df_float['tag'] = tag
        # TODO: should connect price ddate with total shares ddate
        total_shares = df_shares.iloc[0].value
        df_float['value'] = price_per_share * total_shares
        df_float['qtrs'] = 0
        df_float['srcdir'] = 'computed'
        df_float['version'] = df_float['footnote'] = pd.NA
        df_float = df_float.astype(df.drop(columns=['fy','fp']).dtypes.to_dict())
        new_df = new_df.append(df_float)
    return new_df

def read_dera_table(zf, fy_qtr, tbl):
    """From a local file ZF, read data for the period FY_QTR for the DERA table TBL.
    Return the Dataframe created so that when it is time to create the actual Trino table
    we know what the shape of the data should look like.  The returned DF has all the data
    of the specific ingestion, not all the data of all the ingestions of data for TBL."""
    global dera_df
    
    df = pd.read_csv(zf, header=0, sep='\t', dtype='string', keep_default_na=False, nrows = None, engine='c')
    df['srcdir'] = fy_qtr
    df.srcdir = df.srcdir.astype('string')
    
    # df = df.convert_dtypes (infer_objects=False, convert_string=True, convert_integer=False, convert_boolean=False, convert_floating=False)
    # Print the output
    # print(df.dtypes)
    
    if tbl=='sub':
        df.name = df.name.map(lambda x: re.sub(dera_regex, '', x))
        df.name = df.name.astype('string')
        df['LEI'] = df.name.map(gleif_dict)
        df.LEI = df.LEI.astype('string')
        df.cik = df.cik.astype('int32')
        df.loc[df.sic=='', 'sic'] = pd.NA
        df.sic = df.sic.astype('Int16')
        df.loc[df.ein=='', 'ein'] = pd.NA
        df.ein = df.ein.astype('Int64')
        df.wksi = df.wksi.astype('bool')
        # df.wksi = df.wksi.astype('int32')
        df.period = pd.to_datetime(df.period, format='%Y%m%d', utc=True, errors='coerce')
        df.fy = pd.to_datetime(df.fy, format='%Y', utc=True, errors='coerce')
        df.filed = pd.to_datetime(df.filed, format='%Y%m%d', utc=True)
        df.accepted = pd.to_datetime(df.accepted, format='%Y-%m-%d %H:%M:%S', utc=True)
        df.prevrpt = df.prevrpt.astype('bool')
        df.detail = df.detail.astype('bool')
        df.nciks = df.nciks.astype('int16')
        
        cols = df.columns.tolist()
        # Move LEI to a more friendly location in the column order
        cols = cols[0:3] + [cols[-1]] + cols[3:-1]
        df = df[cols]
    elif tbl=='num':
        # documentation wrongly lists coreg as NUMERIC length 256.  It is ALPHANUMERIC.
        if fy_qtr=='2021q3':
            df.loc[df.ddate=='30210630', 'ddate'] = '20210630'
        if fy_qtr=='2019q2':
            df.loc[df.ddate=='29171231', 'ddate'] = '20171231'
        if fy_qtr=='2017q3':
            df.loc[df.ddate=='60160630', 'ddate'] = '20160630'
        # Fix some bad AES data
        if fy_qtr=='2021q1':
            df.loc[(df.adsh=='0000874761-21-000015')&(df.tag=='CommonStockValue')&(df.ddate=='20190630'), 'ddate'] = '20200630'
        elif fy_qtr=='2020q1':
            df.loc[(df.adsh=='0000874761-20-000012')&(df.tag=='EntityPublicFloat')&(df.ddate=='20180630'), 'ddate'] = '20190630'
            df.loc[(df.adsh=='0000874761-20-000012')&(df.tag=='CommonStockValue')&(df.ddate=='20180630'), 'ddate'] = '20190630'
        df.ddate = pd.to_datetime(df.ddate, format='%Y%m%d', utc=True)
        df.qtrs = df.qtrs.astype('int16')
        df.loc[df.coreg=='', 'coreg'] = pd.NA
        df.loc[df.value=='', 'value'] = pd.NA
        df.value = df.value.astype('Float64')
        df.loc[df.footnote=='', 'footnote'] = pd.NA
        
        print(f"Inferring floats: start len(df) = {len(df)}")
        annual_df = dera_df['sub'][dera_df['sub'].form.isin(['10-K','20-F','40-F'])]
        df['fy'] = df.adsh.map(dict(zip(annual_df.adsh,annual_df.fy)))
        df['fp'] = df.adsh.map(dict(zip(annual_df.adsh,annual_df.fp)))
        print(f"len(df[df.fp=='FY']) = {len(df[df.fp=='FY'])}")
        df = df[df.fp=='FY']
        # df = df.assign(cik=df.adsh.str[:10])
        df_has_float = df[df.tag.isin(float_tags)]
        print(f"len(df_has_float) = {len(df_has_float)}")
        df_needs_float = df[~df.adsh.isin(df_has_float.adsh)]
        print(f"len(df_needs_float) = {len(df_needs_float)}")
        float_df = infer_float(df_needs_float[df_needs_float.coreg.isna()
                                              &df_needs_float.tag.isin(all_float_helper_tags)
                                              &(df_needs_float.value>0)].groupby(['adsh'], as_index=False))
        df = df.drop(columns=['fy','fp'])
        # print(df.dtypes)
        # print(float_df.dtypes)
        if float_df.empty:
            print(f"{len(float_df)} floats inferred; Sorry!")
        else:
            float_df = float_df.astype(df.dtypes.to_dict())
            df = pd.concat([df, float_df])
            print(f"{len(float_df)} floats inferred; {len(float_df[float_df.tag=='ComputedTreasuryFloat'])} treasury-based; {len(float_df[float_df.tag=='ComputedMarketFloat'])} market-based")
    elif tbl=='tag':
        df.custom = df.custom.astype('bool')
        df.abstract = df.abstract.astype('bool')
        df.loc[df.crdr=='', 'crdr'] = pd.NA
        df.loc[df.tlabel=='', 'tlabel'] = pd.NA
        df.loc[df.doc=='', 'doc'] = pd.NA
        df.doc = df.doc.str.replace(':', '\:')
    # print(df.dtypes)
    # display(df.head())

    return df

In [13]:
dera_tables = ['tag', 'sub', 'num']

if False:
    for tbl in dera_tables:
        qres = engine.execute(f"delete from {ingest_schema}.dera_{tbl}3")
        print(qres.fetchall())

In [14]:
import io
import zipfile
import datetime

objects=source_bucket.objects.filter(Prefix='SEC-DERA/2017q4')

for obj in objects:
    if obj.key.endswith('.zip'):
        zipfile_src = s3_source.Object(os.environ['S3_LANDING_BUCKET'],obj.key)
        tmpname = obj.key.split('/')[-1]
        zipfile_src.download_file(f'/tmp/{tmpname}')
        zipfile_obj = zipfile.ZipFile(f'/tmp/{tmpname}', mode='r')
        fy_qtr = tmpname.split('.')[0]
        for tbl in dera_tables:
            print(f'{fy_qtr} - {tbl}')
            with zipfile_obj.open(f"{tbl}.txt") as zf:
                # Read data from ZF into a dataframe.
                dera_df[tbl] = read_dera_table (zf, fy_qtr, tbl)
        zipfile_obj.close()

        if False:
            # Alas, there is some minor post-fixing we need to do before ingesting into parquet
            df = dera_df['num']
            num_fields = df.columns
            df = df[df.tag=='ComputedTreasuryFloat']
            treasury_df = dera_df['sub'].loc[dera_df['sub'].fp=='FY', ['adsh', 'cik', 'name','fye', 'fy', ]].merge(df, on='adsh')
            if (len(treasury_df)>0):
                display(treasury_df)
                grouped_df = treasury_df.groupby('cik')
                for key, item in grouped_df:
                    if len(item)==1:
                        df = item[num_fields].copy()
                        df.ddate = pd.to_datetime(f"{item.fy.squeeze().year}1231", format='%Y%m%d')
                        df.version = item.adsh.squeeze()
                        # df = df.astype(dera_df['num'].dtypes.to_dict())
                        print("adding fact (1)")
                        print(df)
                        df = df.astype(dera_df['num'].dtypes.to_dict())
                        dera_df['num'] = dera_df['num'].append(df)
                    else:
                        item = item.sort_values('ddate', ascending=False).reset_index()
                        df = generate_intermediate_ddate_value(item.iloc[0:2])
                        dera_df['num'] = dera_df['num'].append(df)
        if False:
            # Now output as parquet files
            for tbl in dera_tables:
                buf = io.BytesIO()
                dera_df[tbl].to_parquet(path=buf)
                buf.seek(0)
                fy_qtr = dera_df[tbl].iloc[0].srcdir
                trino_bucket.upload_fileobj(Fileobj=buf,
                                            Key=f'trino/{ingest_schema}/{tbl}/{fy_qtr}.parquet')
        else:
            # Import into Trino the slow way...
            for tbl in dera_tables:
                ingest_table = f"dera_{tbl}4"
                partition = {"sub":"'bucket(adsh,20)', 'bucket(cik,20)', 'fy'", "tag":"'bucket(tag,20)'", "num":"'bucket(adsh,20)', 'bucket(tag,20)', 'ddate'"}[tbl]
                columnschema = osc.create_table_schema_pairs(dera_df[tbl],
                                                             typemap={"int16":"integer", "Int16":"integer",
                                                                      "datetime64[ns, UTC]":"timestamp(6)"})
                if partition:
                    partition_string = f"partitioning = array[{partition}],"
                else:
                    partition_string = ""
                tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    {partition_string}
    format = 'ORC'
)
"""
                print(tabledef)
                qres = engine.execute(tabledef)
                print(qres.fetchall())
 
                dera_df[tbl].to_sql(ingest_table,
                                    con=engine, schema=ingest_schema, if_exists='append',
                                    index=False,
                                    method=osc.TrinoBatchInsert(batch_size=1000, verbose = False))

2017q4 - tag
2017q4 - sub
2017q4 - num
Inferring floats: start len(df) = 2281152
len(df[df.fp=='FY']) = 217585
len(df_has_float) = 1047
len(df_needs_float) = 2638
0001213900-17-012184: merge failed (2)
len(df_prices) = 3
len(df_shares) = 1


,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote,srcdir,fy,fp
1897396,0001213900-17-012184,SharePrice,us-gaap/2017,<NA>,2015-06-30 00:00:00+00:00,0,USD,6.0,<NA>,2017q4,2017-01-01 00:00:00+00:00,FY
1897397,0001213900-17-012184,SharePrice,us-gaap/2017,<NA>,2016-06-30 00:00:00+00:00,0,USD,1.71,<NA>,2017q4,2017-01-01 00:00:00+00:00,FY
1897398,0001213900-17-012184,SharePrice,us-gaap/2017,<NA>,2017-06-30 00:00:00+00:00,0,USD,0.51,<NA>,2017q4,2017-01-01 00:00:00+00:00,FY


,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote,srcdir,fy,fp
2600,0001213900-17-012184,EntityCommonStockSharesOutstanding,dei/2014,<NA>,2017-10-31 00:00:00+00:00,0,shares,2411070.0,<NA>,2017q4,2017-01-01 00:00:00+00:00,FY


0 floats inferred; Sorry!

create table if not exists osc_datacommons_dev.sandbox.dera_tag4(
    tag varchar,
    version varchar,
    custom boolean,
    abstract boolean,
    datatype varchar,
    iord varchar,
    crdr varchar,
    tlabel varchar,
    doc varchar,
    srcdir varchar
) with (
    partitioning = array['bucket(tag,20)'],
    format = 'ORC'
)

[(True,)]

create table if not exists osc_datacommons_dev.sandbox.dera_sub4(
    adsh varchar,
    cik integer,
    name varchar,
    LEI varchar,
    sic integer,
    countryba varchar,
    stprba varchar,
    cityba varchar,
    zipba varchar,
    bas1 varchar,
    bas2 varchar,
    baph varchar,
    countryma varchar,
    stprma varchar,
    cityma varchar,
    zipma varchar,
    mas1 varchar,
    mas2 varchar,
    countryinc varchar,
    stprinc varchar,
    ein bigint,
    former varchar,
    changed varchar,
    afs varchar,
    wksi boolean,
    fye varchar,
    form varchar,
    period timestamp(6),
    fy timestamp(6),
  

In [15]:
# Borrow metadata code from DERA-iceberg if/when we need it

In [16]:
tablenames = ['dera_sub4', 'dera_num4', 'dera_tag4', 'ticker']
l = []
for tbl in tablenames:
    qres = engine.execute(f'select count (*) from {ingest_catalog}.{ingest_schema}.{tbl}')
    l.append(qres.fetchall()[0])
print(list(zip(tablenames, l)))

[('dera_sub4', (6550,)), ('dera_num4', (217585,)), ('dera_tag4', (99490,)), ('ticker', (12421,))]
